In [84]:
%pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
%pip install -U -q "google-genai==1.7.0"
%pip install -U -q scikit-learn
%pip install -U -q pandas
%pip install -U -q tqdm rich ipywidgets



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [85]:
from google import genai
from google.genai import types


genai.__version__

'1.7.0'

In [86]:
import os
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
client = genai.Client(api_key=GOOGLE_API_KEY)

In [87]:
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)

models/gemini-1.5-flash-001-tuning


In [88]:
import pprint
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroup_test = fetch_20newsgroups(subset="test")

newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [89]:
print(newsgroups_train.data[0])



From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [90]:
import email
import re

import pandas as pd

def preprocess_newsgroup_row(data):
    msg = email.message_from_string(data)
    text = f"{msg['Subject']}\n\n {msg.get_payload()}"

    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)

    text = text[:4000]
    return text


def preprocess_newsgroup_data(newsgroup_dataset):
    df = pd.DataFrame({
        "Text": newsgroup_dataset.data,
        "Label": newsgroup_dataset.target
    })

    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)

    df["Class Name"] = df["Label"].map(lambda l: newsgroup_dataset.target_names[l])

    return df




In [91]:
df_train = preprocess_newsgroup_data(newsgroups_train)
df_test = preprocess_newsgroup_data(newsgroup_test)

df_train.head()

,Text,Label,Class Name
0,WHAT car is this!?\n\n I was wondering if any...,7,rec.autos
1,SI Clock Poll - Final Call\n\n A fair number o...,4,comp.sys.mac.hardware
2,"PB questions...\n\n well folks, my mac plus fi...",4,comp.sys.mac.hardware
3,Re: Weitek P9000 ?\n\n Robert J.C. Kyanko () w...,1,comp.graphics
4,Re: Shuttle Launch Question\n\n From article <...,14,sci.space


In [92]:
def sample_data(df, num_samples, classes_to_keep):
    # Sample rows, selecting num_samples of each Label.
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)
    )

    df = df[df["Class Name"].str.contains(classes_to_keep)]
    df["Class Name"] = df["Class Name"].astype("category")

    return df


TRAIN_NUM_SAMPLES = 50
TEST_NUM_SAMPLES = 10
# Keep rec.* and sci.*
CLASSES_TO_KEEP = "^rec|^sci"

df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)
df_train.head()

,Text,Label,Class Name
350,"Re: Instead of a Saturn SC2, What???\n\n In <I...",7,rec.autos
351,BRAKE ROTORS...CROSS DRILLING...312 702 8323\n...,7,rec.autos
352,Re: $6700 for hail damage - a record?\n\n I wa...,7,rec.autos
353,MILITECH\n\n I saw an interesting product in N...,7,rec.autos
354,"Re: I'm getting a car, I need opinions.\n\n In...",7,rec.autos


In [93]:
sample_idx = 0
sample_row = preprocess_newsgroup_row(newsgroup_test.data[sample_idx])

sample_label = newsgroup_test.target_names[newsgroup_test.target[sample_idx]]
print(sample_row)
print('---')
print('Label:', sample_label)

Need info on 88-89 Bonneville

 
 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

---
Label: rec.autos


In [94]:
response = client.models.generate_content(
    model="gemini-1.5-flash-001",
    contents=sample_row
)

print(response.text)

You're right to be a little confused!  The 1988-1989 Pontiac Bonnevilles had a lot of different trim levels and options. Here's a breakdown of the models and some general information about value:

**Model Breakdown:**

* **Base Bonneville:**  The basic model came with a 3.8L V6 engine, power steering, power brakes, and other standard features.
* **LE:**  The LE stood for "Luxury Edition" and added features like cloth upholstery, a rear window defroster, and an AM/FM cassette stereo.
* **SE:**  The SE stood for "Sport Edition" and offered sportier styling with a blacked-out grille, a different wheel design, and often a slightly more powerful engine option (sometimes a 3.8L V6 with a higher horsepower rating or a 5.0L V8).
* **LSE:**  The LSE combined the features of the LE and the SE.  Think of it as a "Luxury Sport Edition."
* **SSE:**  The SSE, or "Sport Sedan Edition," was the top-of-the-line model. It came with a 5.0L V8 engine, a sport suspension, leather upholstery, and a host of 

In [95]:
prompt = "From what newsgroup dose the following message originate ?"
baseline_response = client.models.generate_content(
    model="gemini-1.5-flash-001",
    contents=[prompt, sample_row]
) 
print(baseline_response.text)

This message is likely from a newsgroup dedicated to **Pontiac Bonnevilles**, most probably **alt.autos.pontiac**. 

Here's why:

* **Subject:** The subject line "Need info on 88-89 Bonneville" clearly indicates the focus is on Pontiac Bonnevilles from those specific years. 
* **Content:** The message discusses various Bonneville trims (LE, SE, LSE, SSE, SSEi) and their differences, which are specific to the Bonneville model. 
* **Time of Year:** The reference to "mid-spring early summer" as the best time to buy is common advice for car purchases, suggesting a focus on a specific vehicle market. 

While the newsgroup name isn't explicitly mentioned in the message, the context strongly points towards a Pontiac Bonneville-specific group. 



In [104]:
from google.api_core import retry

system_instruct = """
Your are classification service. You will be passed input that represents newsgroup  post and you must respond with the newsgroup from which the post originates.
"""

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429,503})

@retry.Retry(predicate=is_retriable)
def predict_label(post: str)->str:
    response = client.models.generate_content(
        model="gemini-1.5-flash-001",
        config=types.GenerateContentConfig(
            system_instruction=system_instruct
        ),
        contents=post
    )

    rc= response.candidates[0]


    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return response.text.strip()
    
prediction = predict_label(sample_row)
print(prediction)
print({
    prediction,
    sample_label
})
print("Correct!" if prediction == sample_label else "Incorrect.")



rec.autos.misc
{'rec.autos', 'rec.autos.misc'}
Incorrect.


In [97]:
import tqdm
from tqdm.rich import tqdm as tqdmr
import warnings

# Enable tqdm features on Pandas.
tqdmr.pandas()

# But suppress the experimental warning
warnings.filterwarnings("ignore", category=tqdm.TqdmExperimentalWarning)


# Further sample the test data to be mindful of the free-tier quota.
df_baseline_eval = sample_data(df_test, 2, '.*')

# Make predictions using the sampled data.
df_baseline_eval['Prediction'] = df_baseline_eval['Text'].progress_apply(predict_label)

# And calculate the accuracy.
accuracy = (df_baseline_eval["Class Name"] == df_baseline_eval["Prediction"]).sum() / len(df_baseline_eval)
print(f"Accuracy: {accuracy:.2%}")

Output()

Accuracy: 31.25%


In [98]:
df_baseline_eval

,Text,Label,Class Name,Prediction
0,Re: Best Radar Detector - VALENTINE-1?\n <1r3g...,7,rec.autos,rec.autos.driving
1,"Re: what is the deal with ""blue-book"" value of...",7,rec.autos,rec.autos
2,Re: Maxima Chain wax\n\n In article <> (Paul ...,8,rec.motorcycles,rec.motorcycles
3,Re: DoD Pins...NOT!\n\n In article <> writes:...,8,rec.motorcycles,rec.motorcycles
4,Re: Professional Sports on FM Radio\n\n In art...,9,rec.sport.baseball,rec.sport.hockey
5,Re: Baseball logos available via anonymous FTP...,9,rec.sport.baseball,rec.sport.baseball
6,"HABS WIN, HABS WIN!!!\n\n On April 23, JBE5 <...",10,rec.sport.hockey,hockey.
7,Re: If You Were Pat Burns ...\n\n If I were Pa...,10,rec.sport.hockey,hockey
8,Re: Is key escrow enough?\n\n The points rai...,11,sci.crypt,(error)
9,Re: freely distributable public key cryptograp...,11,sci.crypt,(error)


In [99]:
from collections.abc import Iterable
import random
input_data = {'examples': 
    df_train[['Text', 'Class Name']].rename(
        columns={"Text": "textInput", "Class Name": "output"}
    ).to_dict(orient="records")
 }

model_id = None

if not model_id:
    queued_model = None
    for m in reversed(client.tunings.list()):
        if m.name.startswith('tunedModels/newsgroup-classification-model'):
            if m.state.name == "JOB_STATE_SUCCEEDED":
                model_id = m.name
                print("Found existing tuned model to reuse.")
                break
            elif m.state.name == "JOB_STATE_RUNNING" and not queued_model:
                queued_model = m.name
        else:
            if queued_model: 
                model_id = queued_model
                print("Found queued model, still waiting.")

if not model_id:
    tuning_op = client.tunings.tune(
        base_model="models/gemini-1.5-flash-001-tuning",
        training_dataset=input_data,
        config=types.CreateTuningJobConfig(
            tuned_model_display_name="Newsgroup classification model",
            batch_size=16,
            epoch_count=2
        )
    )

    print(tuning_op.state)
    model_id = tuning_op.name
print(model_id)


Found existing tuned model to reuse.
tunedModels/newsgroup-classification-model-3przllsj9


In [100]:
import datetime
import time

MAX_WAIT = datetime.timedelta(minutes=10)

while not (tuned_model := client.tunings.get(name=model_id)).has_ended:
    print(tuned_model.state)
    time.sleep(60)

    if datetime.datetime.now(datetime.timezone.utc) - tuned_model.create_time > MAX_WAIT:
        print("Taking a shortcut, using a previously prepared model.")
        model_id = "tunedModels/newsgroup-classification-model-ltenbi1b"
        tuned_model = client.tunings.get(name=model_id)
        break

print(f"Done! the model state is: {tuned_model.state.name}")
if not tuned_model.has_succeeded and tuned_model.error:
    print("Error:", tuned_model.error)


Done! the model state is: JOB_STATE_SUCCEEDED


In [101]:
new_text = """
First-timer looking to get out of here.

Hi, I'm writing about my interest in travelling to the outer limits!

What kind of craft can I buy? What is easiest to access from this 3rd rock?

Let me know how to do that please.
"""

response = client.models.generate_content(
    model=model_id, contents=new_text)

print(response.text)


sci.space


In [106]:
@retry.Retry(predicate=is_retriable)
def classify_text(text: str) -> str:
    """Classify the provided text into a known newsgroup."""
    response = client.models.generate_content(
        model=model_id, contents=text)
    rc = response.candidates[0]

    # Any errors, filters, recitation, etc we can mark as a general error
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return rc.content.parts[0].text


# The sampling here is just to minimise your quota usage. If you can, you should
# evaluate the whole test set with `df_model_eval = df_test.copy()`.
df_model_eval = sample_data(df_test, 4, '.*')

df_model_eval["Prediction"] = df_model_eval["Text"].progress_apply(classify_text)

accuracy = (df_model_eval["Class Name"] == df_model_eval["Prediction"]).sum() / len(df_model_eval)
print(f"Accuracy: {accuracy:.2%}")

Output()

ServerError: 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}

In [107]:
sysint_tokens = client.models.count_tokens(
    model="gemini-1.5-flash-001",
    contents=[system_instruct, sample_row]
).total_tokens


print(f'System instructed baseline model: {sysint_tokens} (input)')
tuned_token  = client.models.count_tokens(model=tuned_model.base_model,contents=sample_row,).total_tokens

savings = (sysint_tokens-tuned_token)/tuned_token
print(f'Token savings: {savings:.2%}')  # Note that this is only n=1.



System instructed baseline model: 170 (input)
Token savings: 23.19%
